1 . 이미지분류 모델을 생성

2 . 모델을 이용해서 폴더에 있는 영상을 각 폴더로 분류하는 기능

In [1]:
# 1. 이미지 분류모델을 생성

In [2]:
# c c p d c c p d flatten fd d fc
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense, Conv2D,MaxPool2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
import os

In [3]:
(X_train,y_train),(X_test,y_test)= tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


In [4]:
X_train_scaled = X_train / 255. ; X_test_scaled = X_test / 255.

In [5]:
model = Sequential([
  Conv2D(64,3,activation='relu',input_shape=(32,32,3)),
  Conv2D(128,3,activation='relu'),
  MaxPool2D(),
  Dropout(0.4),
  Conv2D(128,3,activation='relu'),
  Conv2D(128,3,activation='relu'),
  MaxPool2D(),
  Dropout(0.4),
  Flatten(),
  Dense(100,activation='relu'),
  Dropout(0.4),
  Dense(10,activation='softmax')
])


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 14, 14, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 128)       147584    
                                                        

In [ ]:
model.compile(optimizer='adam',metrics=['acc'],loss='sparse_categorical_crossentropy')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best_cnn.keras',save_best_only=True)
ealy_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
hist = model.fit(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),callbacks=[ealy_stopping_cb,checkpoint_cb], epochs=20)

In [15]:
from zipfile import ZipFile

In [16]:
with ZipFile('/content/test_images.zip','r') as f:
  f.extractall("./")

In [17]:
class_name = ['airplane','automobile','bird','cat','deer','dog','flog','horse','ship','truck']

In [21]:
images_path = '/content/'
from glob import glob
from PIL import Image
import numpy as np

In [34]:
x_test = []; original_files = []
for filepath in glob("./*.jpg"):
  original_files.append(filepath)
  img = Image.open(filepath)
  x = np.array(img.resize([32,32])) / 255.
  x_test.append(x)
x_test = np.array(x_test)

In [35]:
x_test.shape

(12, 32, 32, 3)

In [37]:
import os
import shutil

In [38]:
output_dir = 'img_category'
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

pred = model.predict(x_test)

# 예측한 데이터를 순회
for idx,preddata in enumerate(pred):
  index = np.argmax(preddata) # 10개의 클래스에 대한 확률 가장큰 확률의 인덱스
  classname = class_name[index]  # 가장큰 인덱스에 해당하는 클래스명
  target_dif_file = output_dir+'/'+ classname
  if not os.path.exists( target_dif_file): # 클래스명으로 디렉터리 생성
    os.makedirs(target_dif_file)
  shutil.copy(original_files[idx] ,target_dif_file )  # 해당 디렉터리로 데이터를 이동


1/1 [==============================] - 0s 20ms/step


RsNet50모델을 학습없이 사용해 보기

In [50]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
base_model = ResNet50(include_top=False,input_shape=(32, 32,3))
# base_model.trainable = False # 가중치 고정
cnn =Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024,activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(10,activation='softmax'))

In [51]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # 가중치를 새로운데이터로 학습하면서 설정한 값을 사용-> 미세조정
cnn.compile(optimizer=adam,metrics=['acc'],loss='sparse_categorical_crossentropy')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best_ReNet50.keras',save_best_only=True)
ealy_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
hist = cnn.fit(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),callbacks=[ealy_stopping_cb,checkpoint_cb], epochs=20)

Epoch 1/20
1563/1563 [==============================] - 129s 50ms/step - loss: 1.2625 - acc: 0.5784 - val_loss: 0.7761 - val_acc: 0.7375
Epoch 2/20
1563/1563 [==============================] - 78s 50ms/step - loss: 0.7376 - acc: 0.7508 - val_loss: 0.6633 - val_acc: 0.7789
Epoch 3/20
1563/1563 [==============================] - 74s 47ms/step - loss: 0.5536 - acc: 0.8125 - val_loss: 0.6727 - val_acc: 0.7787
Epoch 4/20
1563/1563 [==============================] - 80s 51ms/step - loss: 0.4400 - acc: 0.8504 - val_loss: 0.6128 - val_acc: 0.8030
Epoch 5/20
1563/1563 [==============================] - 72s 46ms/step - loss: 0.3414 - acc: 0.8841 - val_loss: 0.6222 - val_acc: 0.8085
Epoch 6/20
1563/1563 [==============================] - 74s 48ms/step - loss: 0.2720 - acc: 0.9074 - val_loss: 0.7711 - val_acc: 0.7873
Epoch 7/20
1563/1563 [==============================] - 72s 46ms/step - loss: 0.2095 - acc: 0.9301 - val_loss: 0.6597 - val_acc: 0.8168


In [53]:
import os
import shutil

x_test = []; original_files = []
for filepath in glob("./*.jpg"):
  original_files.append(filepath)
  img = Image.open(filepath)
  x = np.array(img.resize([32,32])) / 255.
  x_test.append(x)
x_test = np.array(x_test)


output_dir = 'img_category4'
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

pred = cnn.predict(x_test)

# 예측한 데이터를 순회
for idx,preddata in enumerate(pred):
  index = np.argmax(preddata) # 10개의 클래스에 대한 확률 가장큰 확률의 인덱스
  classname = class_name[index]  # 가장큰 인덱스에 해당하는 클래스명
  target_dif_file = output_dir+'/'+ classname
  if not os.path.exists( target_dif_file): # 클래스명으로 디렉터리 생성
    os.makedirs(target_dif_file)
  shutil.copy(original_files[idx] ,target_dif_file )  # 해당 디렉터리로 데이터를 이동

1/1 [==============================] - 1s 853ms/step
